<a href="https://colab.research.google.com/github/ujeongmin/ESAA/blob/main/ESAA_HW_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

(파이썬 머신러닝 완벽 가이드)
## [CH8 텍스트 분석]
NLP는 머신이 인간의 언어를 이해하고 해석하는 데 중점, 텍스트 마이닝은 비정형 텍스트에서 의미 있는 정보를 추출하는 것에 중점

ex) NLP - 번역, 질의응답 시스템 등 / 텍스트 분석 - 머신러닝, 비즈니스 인텔리전스, 예측 분석 등


### 01. 텍스트 분석 이해
머신러닝 알고리즘은 숫자형의 피처 기반 데이터만 입력받을 수 있음  
텍스트를 word 기반의 다수의 피처로 추출하고 이 피처에 단어 빈도수와 같은 숫자값을 부여 -> 단어의 조합인 벡터값으로 표현될 수 있음
* BOW(Bag of Words) 방법

텍스트 분석 수행 프로세스 : 텍스트 전처리 -> 피처 벡터화/추출 -> ML 모델 수립 및 학습/예측/평가

파이썬 기반의 NLP, 텍스트 분석 패키지 : NLTK, Gensim, SpaCy



### 02. 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화
#### 클렌징
텍스트에서 분석에 방해가 되는 불필요한 문자, 기호등을 사전에 제거 (ex) HTML, XML 태그, 기호

#### 텍스트 토큰화
문서에서 문장을 분리하는 문장 토큰화, 문장에서 단어를 분리하는 단어 토큰화

* 문장 토큰화

In [1]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. You can see it out your window or on your television. You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)
print(type(sentences), len(sentences))
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


* sent_tokenize()는 문장으로 구성된 list 반환

* 단어 토큰화  

기본적으로 공백, 콤마, 마침표 등으로 단어 분리  
BOW와 같이 단어의 순서가 중요하지 않은 경우 문장 토큰화를 사용하지 않고 단어 토큰화만 사용해도 충분

In [2]:
from nltk import word_tokenize

sentence = 'The Matrix is everywhere its all around us, here even in this room.'
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [3]:
from nltk import word_tokenize, sent_tokenize

# 여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성
def tokenize_text(text):
  # 문장별로 분리 토큰
  sentences = sent_tokenize(text)
  # 분리된 문장별 단어 토큰화
  word_tokens = [word_tokenize(sentence) for sentence in sentences]
  return word_tokens

# 여러 문장에 대해 문장별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


* 문장을 단어별로 하나씩 토큰화 할 경우 문맥적인 의미는 무시될 수 밖에 없음
* 이를 조금이라도 해결하기 위해 n-gram 도입
* n-gram : 연속된 n개의 단어를 하나의 토큰화 단위로 분리, n개 단어 크기 윈도우를 만들어 문장의 처음부터 오른쪽으로 움직이면서 토큰화 수행

#### 스톱 워드 제거
분석에 큰 의미가 없는 단어 제거 (ex) is, the, a, will 등
* NTLK에 스톱워드가 언어별로 목록화되어 있음

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
print('영어 stop words 개수:', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


* 영어는 스톱 워드가 179개

In [8]:
import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
# 위 예제에서 3개의 문장별로 얻은 word_tokens list에 대해 스톱 워드를 제거하는 반복문
for sentence in word_tokens:
  filtered_words = []
  # 개별 문장별로 토큰화된 문장 list에 대해 스톱 워드를 제거하는 반복문
  for word in sentence:
    # 소문자로 모두 변환
    word = word.lower()
    # 토큰화된 개별 단어가 스톱 워드의 단어에 포함되지 않으면 word_tokens에 추가
    if word not in stopwords:
      filtered_words.append(word)
  all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


#### Stemming과 Lemmatization
문법적인 요소에 따라 단어가 다양하게 변하는데 (과거, 현재, 3인칭, 진행형 등) Stemming과 Lemmatization은 문법적 또는 의미적으로 변화하는 단어의 원형을 찾음

Stemming: 원형 단어로 변환시 일반적인 방법을 적용하거나 더 단순화된 방법을 적용  
Lemmatization: 문법적인 요소와 더 의미적인 부분을 감안해 정확한 철자로 된 어근 단어 찾음 (더 오래 걸림)

In [11]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'), stemmer.stem('works'), stemmer.stem('worked'))
print(stemmer.stem('amusing'), stemmer.stem('amuses'), stemmer.stem('amused'))
print(stemmer.stem('happier'), stemmer.stem('happiest'))
print(stemmer.stem('fancier'), stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


* work는 단순히 뒤에 붙는 변화만 있어서 제대로 인식하지만, amuse, happy, fancy는 원형을 잘 찾지 못함

In [12]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing', 'v'), lemma.lemmatize('amuses', 'v'), lemma.lemmatize('amused', 'v'))
print(lemma.lemmatize('happier', 'a'), lemma.lemmatize('happiest', 'a'))
print(lemma.lemmatize('fancier', 'a'), lemma.lemmatize('fanciest', 'a'))

[nltk_data] Downloading package wordnet to /root/nltk_data...


amuse amuse amuse
happy happy
fancy fancy


* Stemmer보다 정확하게 원형 단어 추출해줌

### 03. Bag of Words - BOW
문서가 가지는 모든 단어를 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델  
쉽고 빠르지만 문맥 의미 반영 부족, 희소 행렬 문제 등의 단점 존재

#### BOW 피처 벡터화
ML 알고리즘은 일반적으로 숫자형 피처를 데이터로 입력받기 때문에 텍스트는 특정 의미를 가지는 숫자형 값인 벡터 값으로 변환해야 함
* 피처 벡터화 2가지 방법  
카운트 기반의 벡터화 : 문서에서 단어 등장 횟수 Count    
TF-IDF 기반의 벡터화 : 개별 문서에서 자주 나타나는 단어에 가중치, 모든 문서 전반적으로 자주 나타나는 단어는 패널티 부여 (더 좋은 예측 성능)

#### 사이킷런의 Count 및 TF-IDF 벡터화 구현: CountVectorizer, TfidfVectorizer
CountVectorizer 클래스는 소문자 일괄 변환, 토큰화, 스톱 워드 필터링 등의 텍스트 전처리도 함께 수행함
* 입력 파라미터 : max_df, min_df, max_features, stop_words, n_gram_range 등
* 절차 : 모두 소문자 변환 등의 전처리 작업 > 단어 기준으로 n_gram_range를 반영해 각 단어를 토큰화 > 텍스트 정규화 > 입력 파라미터를 이용해 토큰화된 단어를 피처로 추출하고 단어 빈도수 벡터 값을 적용(피처 벡터화)  

TF_IDF 벡터화는 TfidfVectorizer 클래스 이용, 파라미터와 변환 방법은 위와 동일


#### BOW 벡터화를 위한 희소 행렬
위 클래스를 이용해 벡터화하고 CSR 형태의 희소 행렬을 반환  
대규모 행렬의 대부분의 값을 0이 차지하는 행렬을 희소 행렬이라고 함

#### 희소 행렬 - COO 형식
0이 아닌 데이터만 별도의 데이터 배열에 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식
* Scipy의 sparse 패키지 coo_matrix 클래스

In [13]:
import numpy as np

dense = np.array([[3,0,1],[0,2,0]])

In [14]:
from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3,1,2])

# 행 위치와 열 위치를 각각 배열로 생성
row_pos = np.array([0,0,1])
col_pos = np.array([0,2,1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

* sparse_coo는 COO 형식의 희소 행렬 객체 변수, 이를 toarray() 메서드를 이용해 다시 밀집 형태로 행렬로 출력

In [15]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

#### 희소 행렬 - CSR 형식
COO 형식이 행과 열의 위치를 나타내기 위해서 반복적인 위치 데이터를 사용해야 하는 문제점을 해결한 방식  
고유 값의 시작 위치만 알고 있으면 얼마든지 행 위치 배열을 다시 만들 수 있기에 메모리가 적게 들고 빠른 연산 가능
* csr_matrix 클래스 이용

In [16]:
from scipy import sparse

dense2 = np.array([[0,0,1,0,0,5],
                   [1,4,0,3,2,5],
                   [0,6,0,3,0,0],
                   [2,0,0,0,0,0],
                   [0,0,0,7,0,8],
                   [1,0,0,0,0,0]])

# 0이 아닌 데이터 추출
data2 = np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

# 행 위치와 열 위치를 각각 array로 생성
row_pos = np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos = np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

# 행 위치 배열의 고유한 값의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0,2,7,9,10,12,13])

# CSR 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


* 실제 사용 시에는 다음과 같이 밀집 행렬을 생성 파라미터로 입력하면 COO나 CSR 희소 행렬로 생성

In [17]:
dense3 = np.array([[0,0,1,0,0,5],
                   [1,4,0,3,2,5],
                   [0,6,0,3,0,0],
                   [2,0,0,0,0,0],
                   [0,0,0,7,0,8],
                   [1,0,0,0,0,0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)